# **Churn Server**


<a id="deploy"></a>
### Deploy our serving class using as a serverless function
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

In [1]:
import mlrun
import pandas as pd
import os

In [2]:
# setting environment 
mlrun.set_environment(project='function-marketplace')

> 2021-10-12 07:29:57,497 [info] loaded project function-marketplace from MLRun DB


('function-marketplace', 'v3io:///projects/{{run.project}}/artifacts')

In [3]:
# Stored artifacts path (S3)
data_path = 'https://s3.wasabisys.com/iguazio/data/churn/test_set.csv'
model_url = 'https://s3.wasabisys.com/iguazio/models/churn/xgb_model.pkl'
model_path = os.getcwd() + '/models'

# Downloading the model
! wget -nc -P {model_path} {model_url}

File ‘/User/functions/churn_server/models/xgb_model.pkl’ already there; not retrieving.



In [4]:
# importing the function from the function marketplace
fn = mlrun.import_function("hub://churn_server").apply(mlrun.auto_mount())
fn.spec.build.commands = ['pip install lifelines==0.22.8', 'pip install xgboost==1.3.1']

# Adding the model 
fn.add_model(key='xgb_model', model_path=model_path ,class_name='ChurnModel')

In [5]:
# Getting the data
x_test = pd.read_csv(data_path)
y_test = x_test['labels']
x_test.drop(['labels'],axis=1,inplace=True)
x_test.head()

,gender,senior,partner,deps,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,MonthlyCharges,tenure_map,ISP_1,ISP_2,Contract_1,Contract_2,Payment_1,Payment_2,Payment_3
0,0,0,1,0,27,1,0,1,0,0,...,1,101.90,2.0,1,0,1,0,1,0,0
1,0,1,0,0,1,1,1,0,0,0,...,1,85.70,0.0,1,0,0,0,0,1,0
2,1,0,0,0,1,1,0,0,0,0,...,1,69.55,0.0,1,0,0,0,0,1,0
3,0,0,0,0,53,1,1,0,1,1,...,0,105.55,4.0,1,0,0,1,0,1,0
4,0,0,0,0,43,1,1,0,1,1,...,1,104.60,3.0,1,0,0,1,0,1,0


### **Testing the model locally**

In [6]:
# an old version of lifelines and xgboost is required when running locally, uncomment to install
# !pip install lifelines==0.22.8
# !pip install xgboost==1.3.1

In [7]:
# importing the class (a must when using mock_to_server)
from churn_server import *

server = fn.to_mock_server()
response = server.test("/v2/models/xgb_model/predict",body={'inputs' : x_test.values.tolist()})

# Calculating model's accuracy
accuracy = sum(1 for x,y in zip(response['outputs'],y_test) if x == y) / len(y_test)
print(f"model's accuracy : {accuracy}")

> 2021-10-12 07:30:00,756 [info] model xgb_model was loaded
> 2021-10-12 07:30:00,757 [info] Initializing endpoint records
> 2021-10-12 07:30:00,768 [info] Loaded ['xgb_model']
model's accuracy : 0.7913907284768212


### **Deploying and testing our model server using HTTP request**

In [9]:
address = fn.deploy()

> 2021-10-12 07:33:32,143 [info] Starting remote function deploy
2021-10-12 07:33:32  (info) Deploying function
2021-10-12 07:33:32  (info) Building
2021-10-12 07:33:32  (info) Staging files and preparing base images
2021-10-12 07:33:32  (info) Building processor image
2021-10-12 07:33:33  (info) Build complete
2021-10-12 07:33:41  (info) Function deploy complete
> 2021-10-12 07:33:42,050 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-churn-server.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.app-lab-eks-testing.iguazio-cd1.com:31334']}


We invoke our model serving function using test data, the data vector is specified in the `inputs` attribute.

In [19]:
import json
import requests

# KFServing protocol event
event_data = {"inputs": x_test.values.tolist()}

# sending data in a json format
response = requests.put(address + "/v2/models/xgb_model/predict", json=json.dumps(event_data))

# returned data is a string 
y_predict = json.loads(response.text)['outputs']
accuracy = sum(1 for x,y in zip(y_predict,y_test) if x == y) / len(y_test)
print(f"model's accuracy : {accuracy}")

model's accuracy : 0.7913907284768212


**[back to top](#top)**